In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import matplotlib_inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from gensim.models import Word2Vec
warnings.filterwarnings('ignore')
import gensim.downloader
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
rus = "russian"
sw = stopwords.words(rus)
morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)
vectorizer = CountVectorizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#ф-я для вывода текстовых данных в удобном формате
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)
#ф-я для удаления символов пунктуации
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in
    string.punctuation])
#ф-я для удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
#ф-я для удаления последовательностей
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text
#ф-я для удаления не буквенных символов
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text)
#ф-я токенизации с использованием Snowball

#ф-я токенизации с помощью PyMorphy2
def tokenize_morphy(text):
    sw = stopwords.words(rus)
    # добавил слово "это", "мочь", "свой", "который", "решить", "сказать"
    sw += ["это", "мочь", "свой", "который", "решить", "сказать"]
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = text.lower()
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                all_words.append("")
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        return ' '.join(all_words)
    else:
        return None


In [ ]:
#Загрузка русского датасета и объедниение
#train_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\train.jsonl'
#val_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\val.jsonl'
train_path = '/content/drive/MyDrive/course_work/data/test_ds1/train.jsonl'
val_path = '/content/drive/MyDrive/course_work/data/test_ds1/val.jsonl'
train_data = pd.read_json(train_path, lines=True)
val_data = pd.read_json(val_path, lines=True)
df = pd.concat([train_data, val_data])
df = df.drop(['label', 'hypothesis', 'verb','negation'
,'idx','no_negation'], axis=1)
df.dropna()
df.head()
df['genre'] = df['genre'].replace('kp', 'interfax')

In [ ]:
sw = stopwords.words(rus)
sw += ["это", "мочь", "свой", "который", "решить", "сказать"]

In [ ]:
Y = pd.DataFrame({'genre':[str(genre) for genre in df['genre']]})
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample) for sample in df['premise']]})
X2 = df_morphy.dropna()
batch2 = X2.join(Y, how = 'right')
batch2 = batch2.dropna()

In [ ]:
batch2

,text,genre
0,сумма ущерб составить один тысяча рубль уточня...,interfax
1,перебрасываться словечко скользить глаз мой ...,fiction
2,сказать хотеть выпить кофе казаться ска...,fiction
3,зима показать характер,interfax
4,гуманность бог подобный сценарий предполагать...,fiction
...,...,...
1091,кто нужный нужно забрать сумасшедший дом о...,fiction
1092,равно любить сказать вернуться знат...,fiction
1093,слово помнить лишь собираться выйти дом ...,interfax
1094,допросить гость следователь восстановить карти...,interfax


In [ ]:
with open("/content/drive/MyDrive/course_work/data/labeled_data.txt", "w") as f:
  for label, text in zip( batch2['genre'], batch2['text']):
    f.write("__label__" + label + ' ' + text + '\n')

In [ ]:
def vectorize_text(text):
    return model.get_sentence_vector(text)

In [ ]:
import fasttext

# Параметры обучения
params = {
    'lr': 0.1, # скорость обучения
    'epoch': 25, # количество эпох
    'dim': 100, # размер эмбеддинга
    'ws': 5, # размер окна для обучения слов
    'minCount': 5, # минимальное количество вхождений слова для его учета
    'neg': 5, # количество отрицательных примеров
    'loss': 'ns', # функция потерь (например, 'ns' для нагрузки на обучение слов)
    'bucket': 2000000, # максимальное количество слов в одном контексте
    'minn': 3, # минимальная длина n-граммы
    'maxn': 6, # максимальная длина n-граммы
    'thread': 4, # количество потоков для обучения
}
data_path = '/content/drive/MyDrive/course_work/data/labeled_data.txt'

model = fasttext.train_supervised(data_path, **params)

sentence_vector = model.get_sentence_vector("зима показать характер")

sentence_vector

array([ 2.0516722e-03, -3.7027474e-02, -2.0140011e-02, -6.9577596e-05,
        3.3757225e-02, -9.8519754e-03,  1.6076334e-02,  3.6750238e-02,
        3.1746693e-02,  9.5909797e-03, -5.4441974e-03,  1.5523890e-02,
       -2.1954652e-02,  2.4488445e-02,  7.9675326e-03,  2.8874576e-03,
       -1.7506957e-02,  1.0329815e-02, -2.8108770e-02,  5.8780443e-03,
        2.3463519e-02,  2.7331958e-02, -1.6976168e-03, -2.9913049e-02,
        2.5669143e-03,  1.9773047e-02,  1.5814558e-02,  1.6279494e-02,
       -1.2118732e-02, -1.7618315e-02,  4.5192367e-03,  1.6846019e-03,
       -2.8148167e-02,  6.0243178e-03,  1.5256417e-02, -7.7404426e-03,
        9.1926996e-03, -2.4865784e-02, -3.0524405e-02, -1.0423806e-02,
        2.1581870e-02, -2.6797906e-03, -1.8160053e-02,  1.6244125e-02,
        8.9967120e-03,  7.2235647e-03, -1.0947965e-02, -2.9940046e-02,
        9.5052691e-03, -1.4368603e-02,  1.1006309e-02,  3.5113167e-02,
       -2.3518007e-02,  1.9548353e-02, -3.0116744e-03, -4.3501365e-03,
      

In [ ]:
batch2['vector'] = batch2['text'].apply(vectorize_text)

In [ ]:
batch2

,text,genre,vector
0,сумма ущерб составить один тысяча рубль уточня...,interfax,"[-0.0049973233, -0.05788369, -0.022105854, 0.0..."
1,перебрасываться словечко скользить глаз мой ...,fiction,"[0.033027966, 0.034461312, -0.02668001, -0.011..."
2,сказать хотеть выпить кофе казаться ска...,fiction,"[0.026169607, 0.015697643, -0.026508495, -0.00..."
3,зима показать характер,interfax,"[0.0020516722, -0.037027474, -0.02014001, -6.9..."
4,гуманность бог подобный сценарий предполагать...,fiction,"[0.016551994, -0.0106822, -0.027485318, -0.005..."
...,...,...,...
1091,кто нужный нужно забрать сумасшедший дом о...,fiction,"[0.021217352, 0.0072238604, -0.024400657, -0.0..."
1092,равно любить сказать вернуться знат...,fiction,"[0.04050326, 0.04686357, -0.030806813, -0.0140..."
1093,слово помнить лишь собираться выйти дом ...,interfax,"[0.009661001, -0.023572851, -0.024190612, -0.0..."
1094,допросить гость следователь восстановить карти...,interfax,"[0.0022565615, -0.03565624, -0.020516362, -1.6..."


In [ ]:
vector_df = pd.DataFrame(batch2['vector'].tolist())
vector_df['label'] = batch2['genre']
batch2.drop(columns=['vector'], inplace=True)

In [ ]:
vector_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,-0.004997,-0.057884,-0.022106,0.002949,0.042434,-0.014633,0.011572,0.044426,0.047348,0.016751,...,-0.014477,0.052153,0.011175,0.000750,-0.041054,0.005773,0.015730,0.008634,0.010521,interfax
1,0.033028,0.034461,-0.026680,-0.011302,0.018207,0.006505,0.046454,0.026228,-0.016651,-0.014279,...,-0.010283,0.019049,-0.005748,-0.008196,0.006820,0.010119,0.024487,0.000260,0.068228,fiction
2,0.026170,0.015698,-0.026508,-0.009257,0.024062,0.002131,0.041232,0.031100,-0.003230,-0.008052,...,-0.011526,0.027393,-0.002106,-0.006475,-0.003396,0.009701,0.023127,0.001940,0.057254,fiction
3,0.002052,-0.037027,-0.020140,-0.000070,0.033757,-0.009852,0.016076,0.036750,0.031747,0.009591,...,-0.011860,0.040709,0.007688,-0.001480,-0.028109,0.005437,0.014892,0.005643,0.017334,interfax
4,0.016552,-0.010682,-0.027485,-0.005207,0.033517,-0.003844,0.034007,0.039000,0.015546,0.000603,...,-0.013961,0.039429,0.002475,-0.004070,-0.017811,0.008859,0.022441,0.004260,0.045581,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,0.021217,0.007224,-0.024401,-0.007309,0.024622,0.000217,0.035602,0.031103,0.002409,-0.005317,...,-0.010807,0.028440,-0.000700,-0.005710,-0.007225,0.007779,0.021174,0.001996,0.049489,fiction
1092,0.040503,0.046864,-0.030807,-0.014031,0.017659,0.009795,0.055971,0.028011,-0.024352,-0.019404,...,-0.011046,0.019155,-0.007953,-0.010275,0.012282,0.011468,0.028086,-0.001319,0.081872,fiction
1093,0.009661,-0.023573,-0.024191,-0.002603,0.034007,-0.006609,0.025049,0.038100,0.023604,0.005317,...,-0.012477,0.040920,0.004329,-0.002487,-0.023264,0.007460,0.019324,0.005774,0.032301,interfax
1094,0.002257,-0.035656,-0.020516,-0.000016,0.033814,-0.009198,0.016563,0.036509,0.030993,0.009776,...,-0.012293,0.041085,0.006793,-0.000714,-0.028241,0.005845,0.015513,0.006099,0.019607,interfax


In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def accuracy(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return accuracy_score(y_test, y_reg), accuracy_score(y_test, y_svc), accuracy_score(y_test, y_dtc), accuracy_score(y_test, y_rfc), accuracy_score(y_test, y_gbc), accuracy_score(y_test, y_abc), accuracy_score(y_test, y_knc)

In [ ]:
def precision(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return precision_score(y_test, y_reg, average="binary", pos_label="fiction"), precision_score(y_test, y_svc, average="binary", pos_label="fiction"), precision_score(y_test, y_dtc, average="binary", pos_label="fiction"), precision_score(y_test, y_rfc, average="binary", pos_label="fiction"), precision_score(y_test, y_gbc, average="binary", pos_label="fiction"), precision_score(y_test, y_abc, average="binary", pos_label="fiction"), precision_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def recall(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return recall_score(y_test, y_reg, average="binary", pos_label="fiction"), recall_score(y_test, y_svc, average="binary", pos_label="fiction"), recall_score(y_test, y_dtc, average="binary", pos_label="fiction"), recall_score(y_test, y_rfc, average="binary", pos_label="fiction"), recall_score(y_test, y_gbc, average="binary", pos_label="fiction"), recall_score(y_test, y_abc, average="binary", pos_label="fiction"), recall_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def f1(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return f1_score(y_test, y_reg, average="binary", pos_label="fiction"), f1_score(y_test, y_svc, average="binary", pos_label="fiction"), f1_score(y_test, y_dtc, average="binary", pos_label="fiction"), f1_score(y_test, y_rfc, average="binary", pos_label="fiction"), f1_score(y_test, y_gbc, average="binary", pos_label="fiction"), f1_score(y_test, y_abc, average="binary", pos_label="fiction"), f1_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def statistic(X_train, X_test, y_train, y_test):
  scores = pd.DataFrame(columns = ["REG", "SVC", "DTC", "RFC", "GBC", "ABC", "KNC"],
                     index = ["ACC", "PREC", "REC", "F1"])
  acc = [reg, svc, dtc, rfc, gbc, abc, knc] = accuracy(X_train, X_test, y_train, y_test)
  scores.iloc[0, :] = acc
  prec = [reg, svc, dtc, rfc, gbc, abc, knc] = precision(X_train, X_test, y_train, y_test)
  scores.iloc[1, :] = prec
  rec = [reg, svc, dtc, rfc, gbc, abc, knc] = recall(X_train, X_test, y_train, y_test)
  scores.iloc[2, :] = rec
  f_1 = [reg, svc, dtc, rfc, gbc, abc, knc] = f1(X_train, X_test, y_train, y_test)
  scores.iloc[3, :] = f_1
  return scores


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vector_df.iloc[:, :-1], vector_df['label'], test_size = 0.2, random_state = 42)
statistic(X_train, X_test, y_train, y_test)

,REG,SVC,DTC,RFC,GBC,ABC,KNC
ACC,0.927273,0.931818,0.927273,0.95,0.936364,0.931818,0.945455
PREC,0.9,0.91453,0.929204,0.931624,0.900826,0.91453,0.938596
REC,0.964286,0.955357,0.9375,0.973214,0.973214,0.955357,0.955357
F1,0.931034,0.934498,0.921053,0.947826,0.935622,0.934498,0.946903
